## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-04-18-52-17 +0000,bbc,Narco-sub carrying 1.7 tonnes of cocaine seize...,https://www.bbc.com/news/articles/cm274lmg7m1o...
1,2025-11-04-18-48-24 +0000,nypost,Intel officer who confronted John Brennan hope...,https://nypost.com/2025/11/04/us-news/intel-of...
2,2025-11-04-18-46-08 +0000,bbc,Former shop worker has 'nightmares' over abuse...,https://www.bbc.com/news/videos/cd7rzyr54zjo?a...
3,2025-11-04-18-45-00 +0000,wsj,Archer Daniels Midland Cuts Outlook on Margin ...,https://www.wsj.com/business/earnings/archer-d...
4,2025-11-04-18-43-03 +0000,startribune,Talks to end the government shutdown intensify...,https://www.startribune.com/talks-to-end-the-g...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2304/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
67,trump,42
72,new,19
57,will,16
249,dead,14
129,election,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
70,2025-11-04-16-51-00 +0000,wsj,President Trump said he will block federal foo...,https://www.wsj.com/politics/policy/trump-thre...,125
197,2025-11-04-10-00-00 +0000,wsj,When the Supreme Court hears arguments over Pr...,https://www.wsj.com/politics/policy/whats-at-s...,124
277,2025-11-03-22-40-36 +0000,nypost,Trump tells New York ‘you must vote’ for Andre...,https://nypost.com/2025/11/03/us-news/trump-te...,117
199,2025-11-04-09-30-53 +0000,bbc,Trump backs Cuomo for New York City mayor and ...,https://www.bbc.com/news/articles/c231e284345o...,107
51,2025-11-04-17-31-00 +0000,wsj,Some Tuesday elections will test whether Trump...,https://www.wsj.com/politics/elections/divided...,100


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
70,125,2025-11-04-16-51-00 +0000,wsj,President Trump said he will block federal foo...,https://www.wsj.com/politics/policy/trump-thre...
277,75,2025-11-03-22-40-36 +0000,nypost,Trump tells New York ‘you must vote’ for Andre...,https://nypost.com/2025/11/03/us-news/trump-te...
156,51,2025-11-04-11-44-30 +0000,cbc,"Dick Cheney, powerful and polarizing U.S. vice...",https://www.cbc.ca/news/world/dick-cheney-obit...
35,50,2025-11-04-17-56-00 +0000,wsj,The government pays out $100 billion a year in...,https://www.wsj.com/politics/policy/what-to-kn...
197,42,2025-11-04-10-00-00 +0000,wsj,When the Supreme Court hears arguments over Pr...,https://www.wsj.com/politics/policy/whats-at-s...
59,40,2025-11-04-17-18-25 +0000,cbc,"NYC mayor's race draws attention, but Californ...",https://www.cbc.ca/news/world/us-tuesday-off-y...
100,39,2025-11-04-15-01-47 +0000,nypost,France threatens to block Shein over sale of c...,https://nypost.com/2025/11/04/world-news/franc...
189,39,2025-11-04-10-23-31 +0000,bbc,'Taxes are going up' - BBC decodes Reeves's pr...,https://www.bbc.com/news/videos/cpd2yx6v687o?a...
51,34,2025-11-04-17-31-00 +0000,wsj,Some Tuesday elections will test whether Trump...,https://www.wsj.com/politics/elections/divided...
210,33,2025-11-04-08-43-32 +0000,bbc,Worker dies after partial collapse of medieval...,https://www.bbc.com/news/articles/ce3kz7jxe0po...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
